In [1]:
# Reimportar librerías después del reinicio del estado
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split


In [14]:
df= pd.read_csv("/Users/isaromobru/Desktop/FakeNews 10.57.24/proyecto_machine_learning_Fake_News/data/processed/archivo.csv")

In [10]:
df

,title,text,subject,date,label,text_length,title_length,text_word_count,title_word_count,subject_grouped
0,U.S. budget fight loom Republicans flip fiscal...,WASHINGTON Reuters head conservative republica...,politicsNews,"December 31, 2017",1,3293,53,469,8,politics
1,U.S. military accept transgender recruit Monda...,WASHINGTON Reuters Transgender people allow fi...,politicsNews,"December 29, 2017",1,2930,56,397,7,politics
2,senior U.S. republican senator let Mr. Mueller...,WASHINGTON Reuters special counsel investigati...,politicsNews,"December 31, 2017",1,1939,50,272,8,politics
3,FBI Russia probe help australian diplomat tip NYT,WASHINGTON Reuters trump campaign adviser Geor...,politicsNews,"December 30, 2017",1,1755,49,241,8,politics
4,trump want Postal Service charge much Amazon s...,SEATTLE WASHINGTON Reuters President Donald Tr...,politicsNews,"December 29, 2017",1,3632,53,530,8,politics
...,...,...,...,...,...,...,...,...,...,...
44684,mcpain John McCain Furious Iran treat US Sailo...,21st Century Wire say 21wire report early week...,Middle-east,"January 16, 2016",0,2291,53,321,9,general
44685,justice Yahoo Settles E mail Privacy Class act...,21st Century Wire say familiar theme whenever ...,Middle-east,"January 16, 2016",0,1102,70,173,14,general
44686,Sunnistan US Allied safe zone plan take Territ...,Patrick Henningsen 21st Century WireRemember...,Middle-east,"January 15, 2016",0,16869,72,2402,11,general
44687,blow $ 700 million Al Jazeera America finally ...,21st Century Wire say Al Jazeera America go hi...,Middle-east,"January 14, 2016",0,1858,55,284,10,general


In [15]:
le = LabelEncoder()
df['subject_encoded'] = le.fit_transform(df['subject_grouped'])


In [16]:
import gc
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Cargar datos (Asegúrate de que tu DataFrame 'df' esté cargado previamente)
df['text'] = df['text'].fillna("")
df['title'] = df['title'].fillna("")

# Vectorizar texto con un número reducido de características
vectorizer_text = TfidfVectorizer(max_features=5000)
X_text = vectorizer_text.fit_transform(df['text'])

vectorizer_title = TfidfVectorizer(max_features=1000)
X_title = vectorizer_title.fit_transform(df['title'])

# Convertir 'subject_encoded' a formato disperso
subject_feature = csr_matrix(df['subject_encoded'].values.reshape(-1, 1))

# **Reducir dimensionalidad con PCA (TruncatedSVD)**
pca = TruncatedSVD(n_components=300)  # Reduce dimensiones a 300 componentes
X_text_reduced = pca.fit_transform(X_text)
X_title_reduced = pca.fit_transform(X_title)

# Combinar las matrices en una única matriz de características
X = hstack([subject_feature, X_title_reduced, X_text_reduced])

# Definir variable objetivo
y = df['label']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Liberar memoria**
del df, X_text, X_title, X_text_reduced, X_title_reduced
gc.collect()

# **Usar un modelo más eficiente**
# Opción 1: Logistic Regression (rápido y eficiente)
model = LogisticRegression()
model.fit(X_train, y_train)

# Opción 2: LightGBM (si necesitas más velocidad)
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

# Evaluación del modelo
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy:.4f}')


[LightGBM] [Info] Number of positive: 16921, number of negative: 18830
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.208752 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 153002
[LightGBM] [Info] Number of data points in the train set: 35751, number of used features: 601
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.473301 -> initscore=-0.106896
[LightGBM] [Info] Start training from score -0.106896
Precisión del modelo: 0.9805


/Users/isaromobru/Desktop/FakeNews 10.57.24/proyecto_machine_learning_Fake_News/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [3]:
le = LabelEncoder()
df['subject_encoded'] = le.fit_transform(df['subject_grouped'])


In [4]:
from scipy.sparse import csr_matrix, hstack
# Eliminar Na
df['text'] = df['text'].fillna("")

# Vectoriza la columna 'text'
vectorizer_text = TfidfVectorizer(max_features=5000)
X_text = vectorizer_text.fit_transform(df['text'])

# Vectoriza la columna 'title'
vectorizer_title = TfidfVectorizer(max_features=1000)
X_title = vectorizer_title.fit_transform(df['title'])

# Convierte la columna 'subject_encoded' a una matriz dispersa
subject_feature = csr_matrix(df['subject_encoded'].values.reshape(-1, 1))

# Combina las tres fuentes de información en una única matriz de características
X = hstack([subject_feature, X_title, X_text])

# Define la variable objetivo; en este ejemplo, se asume que la columna "label" es la etiqueta a predecir
y = df['label']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)  # Reducimos dimensiones a 50
X_reduced = pca.fit_transform(X)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # Limitamos a 5000 palabras
X_tfidf = vectorizer.fit_transform(text_data)


In [5]:

# Definir los modelos y sus hiperparámetros a optimizar
param_grid = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
        }
    }
}

In [6]:
# Realizar la búsqueda en malla para cada modelo
best_models = {}
for name, model_info in param_grid.items():
    print(f"\n🔍 Optimizando {name}...")
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)  # Usamos la representación TF-IDF de los datos
    
    best_models[name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    }
    print(f"✅ Mejor precisión para {name}: {grid_search.best_score_:.4f}")
    print(f"📌 Mejores hiperparámetros: {grid_search.best_params_}")



🔍 Optimizando RandomForest...
✅ Mejor precisión para RandomForest: 0.9981
📌 Mejores hiperparámetros: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}


In [9]:


# Definir los modelos y sus hiperparámetros a optimizar
param_gridxg = {
    
    'XGBoost': {
        'model': XGBClassifier( eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 10],
            'learning_rate': [0.01, 0.1, 0.2],
        }
 
    }
}


In [10]:
# Realizar la búsqueda en malla para cada modelo
best_models1 = {}
for name, model_info in param_gridxg.items():
    print(f"\n🔍 Optimizando {name}...")
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)  # Usamos la representación TF-IDF de los datos
    grid_search.fit(X_train, y_train)  # Usamos la representación TF-IDF de los datos
    
    best_models[name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    }
    print(f"✅ Mejor precisión para {name}: {grid_search.best_score_:.4f}")
    print(f"📌 Mejores hiperparámetros: {grid_search.best_params_}")



🔍 Optimizando XGBoost...


KeyboardInterrupt: 

In [ ]:


# Definir los modelos sus hiperparámetros a optimizar
param_grid = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 10],
            'learning_rate': [0.01, 0.1, 0.2],
        }
 
    }
}

# Realizar la búsqueda en malla para cada modelo
best_models = {}
for name, model_info in param_grid.items():
    print(f"\n🔍 Optimizando {name}...")
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_tfidf, y)  # Usamos la representación TF-IDF de los datos
    
    best_models[name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    }
    print(f"✅ Mejor precisión para {name}: {grid_search.best_score_:.4f}")
    print(f"📌 Mejores hiperparámetros: {grid_search.best_params_}")

# Convertir resultados a DataFrame y mostrar
df_best_models = pd.DataFrame(best_models).T
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Mejores Modelos con GridSearchCV", dataframe=df_best_models)
